In [1]:
import pandas as pd
import os
import glob

# Define o path pattern para todos os CSVs
path_pattern = '../../Data/Data_Exploration/descriptive_stats_*.csv'

# Encontrar todos os CSVs
csv_files = glob.glob(path_pattern)

# Dicionário para armazenar as correlações por variável
correlations_data = {}

# Processar cada CSV
for file_path in csv_files:
    try:
        df = pd.read_csv(file_path)

        filename = os.path.basename(file_path).lower()
        
        # Determinar tipo de transformação
        if 'model' in filename and 'log' in filename:
            col_suffix = 'model_log'
        elif 'model' in filename:
            col_suffix = 'model'
        else:
            print(f"Warning: Could not determine type for {filename}")
            continue

        # Extrair nomes das variáveis e valores apenas das correlações
        for idx, row in df.iterrows():
            variable_name = df.iloc[idx, 0]  # Primeira coluna tem os nomes das variáveis
            
            if variable_name not in correlations_data:
                correlations_data[variable_name] = {'variable': variable_name}
            
            # Correlações com ros_p original
            if 'corr_ros_p' in df.columns and pd.notna(row['corr_ros_p']):
                correlations_data[variable_name][f'corr_ros_p_{col_suffix}'] = row['corr_ros_p']
            
            # Correlações com log_ros_p
            if 'corr_log_ros_p' in df.columns and pd.notna(row['corr_log_ros_p']):
                correlations_data[variable_name][f'corr_log_ros_p_{col_suffix}'] = row['corr_log_ros_p']

    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")

# Criar DataFrame consolidado
if correlations_data:
    correlations_df = pd.DataFrame(correlations_data.values())

    # Colunas desejadas - apenas correlações
    desired_columns = ['variable']
    
    # Adicionar colunas para cada tipo de modelo
    for model_type in ['model', 'model_log']:
        # Apenas correlações
        desired_columns.extend([
            f'corr_ros_p_{model_type}',
            f'corr_log_ros_p_{model_type}'
        ])

    # Adicionar colunas ausentes com NaN
    for col in desired_columns:
        if col not in correlations_df.columns:
            correlations_df[col] = None

    # Reordenar colunas
    correlations_df = correlations_df[desired_columns]

    # Salvar CSV final apenas com correlações
    output_file = '../../Data/Data_Exploration/all_ros_p_correlations.csv'
    correlations_df.to_csv(output_file, index=False)

    print(f"Successfully created {output_file}")
    print(f"Total variables: {len(correlations_df)}")
    print(f"Files processed: {len(csv_files)}")
    
    # Estatísticas resumidas
    print(f"\n=== RESUMO DAS CORRELAÇÕES ===")
    
    for model_type in ['model', 'model_log']:
        print(f"\n--- {model_type.upper().replace('_', ' ')} ---")
        
        # Correlações com ros_p
        corr_col = f'corr_ros_p_{model_type}'
        if corr_col in correlations_df.columns:
            valid_corr = correlations_df[corr_col].dropna()
            if len(valid_corr) > 0:
                print(f"Corr com ros_p: {len(valid_corr)} vars")
                print(f"  Média absoluta: {valid_corr.abs().mean():.3f}")
                print(f"  Máxima absoluta: {valid_corr.abs().max():.3f}")
                print(f"  Mínima absoluta: {valid_corr.abs().min():.3f}")
        
        # Correlações com log_ros_p
        corr_log_col = f'corr_log_ros_p_{model_type}'
        if corr_log_col in correlations_df.columns:
            valid_corr_log = correlations_df[corr_log_col].dropna()
            if len(valid_corr_log) > 0:
                print(f"Corr com log_ros_p: {len(valid_corr_log)} vars")
                print(f"  Média absoluta: {valid_corr_log.abs().mean():.3f}")
                print(f"  Máxima absoluta: {valid_corr_log.abs().max():.3f}")
                print(f"  Mínima absoluta: {valid_corr_log.abs().min():.3f}")

    print(f"\nPrimeiras 10 linhas do DataFrame:")
    print(correlations_df.head(10))

else:
    print("No correlation data found in any of the files")

Successfully created ../../Data/Data_Exploration/all_ros_p_correlations.csv
Total variables: 103
Files processed: 2

=== RESUMO DAS CORRELAÇÕES ===

--- MODEL ---
Corr com ros_p: 102 vars
  Média absoluta: 0.149
  Máxima absoluta: 0.629
  Mínima absoluta: 0.001
Corr com log_ros_p: 102 vars
  Média absoluta: 0.161
  Máxima absoluta: 0.557
  Mínima absoluta: 0.001

--- MODEL LOG ---
Corr com ros_p: 102 vars
  Média absoluta: 0.162
  Máxima absoluta: 0.596
  Mínima absoluta: 0.005
Corr com log_ros_p: 102 vars
  Média absoluta: 0.158
  Máxima absoluta: 0.583
  Mínima absoluta: 0.001

Primeiras 10 linhas do DataFrame:
     variable  corr_ros_p_model  corr_log_ros_p_model  corr_ros_p_model_log  \
0  1_3y_fir_p         -0.058235             -0.094051             -0.045531   
1  3_8y_fir_p          0.001567             -0.000769              0.073072   
2  8_ny_fir_p         -0.055394             -0.074133              0.016197   
3    BLH_m_av          0.154395              0.265683          

In [2]:
# Parâmetros
top_n = 15

# Para cada coluna de correlação
for col in ['corr_ros_p_model', 'corr_log_ros_p_model', 'corr_ros_p_model_log', 'corr_log_ros_p_model_log']:
    if col in correlations_df.columns:
        # Remover NaN e ordenar por valor absoluto
        df_filtrado = correlations_df.dropna(subset=[col])
        top_vars = df_filtrado.reindex(df_filtrado[col].abs().sort_values(ascending=False).index).head(top_n)
        
        print(f"\nTop {top_n} |{col}|:")
        print(top_vars[['variable', col]])


Top 15 |corr_ros_p_model|:
       variable  corr_ros_p_model
58    ros_p_lg1          0.628792
100   wv_850_av          0.378132
93     wv100_av          0.335135
94   wv100_k_av          0.328840
81     wSv_1_av          0.326902
14   FWI_12h_av          0.320707
31   duration_p         -0.314464
48       inidoy          0.305191
11    DC_12h_av          0.304586
101   wv_950_av          0.304578
33       enddoy          0.303759
15       HDW_av          0.287143
102    wv_Fb_av          0.277767
99    wv_700_av          0.276517
29    d_2m_C_av         -0.266519

Top 15 |corr_log_ros_p_model|:
       variable  corr_log_ros_p_model
58    ros_p_lg1              0.557336
31   duration_p             -0.428094
15       HDW_av              0.383461
93     wv100_av              0.359288
51     rh_2m_av             -0.354920
94   wv100_k_av              0.346269
30      dfmc_av             -0.346212
14   FWI_12h_av              0.329779
102    wv_Fb_av              0.329367
95      wv10_av 